# Propagation Exercise 2

In the last exercise, we were using an acoustic modem to send simple commands to an AUV. Now we want to be able to reprogram the AUV while it is deployed so we are going to need send lots of information to the AUV. To do this, we are going to have to encode each character in our new program into a unique pulse that gets transmitted to the AUV. This introduces some new limitations to how we operate the modem:

1. As before, the receiver can't detect and read the signal if the signal level is close to the background noise level.
2. The receiver can't detect and read the signal if the signal is interfering with itself. This means that the direct path signal can't interfere with any of the reflected paths.

**The goal of this exercise is to determine how close the AUV has to be for us to send our new program and to determine the pulse rate we can use to transmit it.**

## Determining the maximum transmission range

Use the widget below to determine the range at which the direct path and the surface reflected path first begin to interfere.

Hint: 
- As you change the range, change the time scale to allow you to zoom in and see the direct and reflected arrivals more clearly.

## Determine the pulse rate of the modem

Determine how much time is needed between pulses so that the direct arrival for the second pulse won't interfere with (15 dB above) the reflected arrivals from the first pulse in a "fine sand" environment.

Hint:
- Increase the number of rays to make sure that all of the reflected arrivals that are within 15 dB of the direct signal are present.
- Adjust the intensity scale and time scale to estimate the time needed for the reflected arrivals to fall below 15 dB.

In [7]:
# Import packages
import numpy as np
import matplotlib.pyplot as plt
from math import ceil, log, pi, atan, sqrt
from scipy.signal.windows import tukey
import ipywidgets as widgets

# Set inline
%matplotlib inline

In [8]:
def CW_pulse_propagation(p, t, tv, frequency, tau, win):
    """
    Function to compute continous wave pulse using Tukey window.
    """
    # Compute the full pulse with the phase factor:
    CW_pulse = p * np.exp(-1j * (2*np.pi*frequency*t - np.pi/2))
    
    # Identify indices where the pulse is active: (t - tv) in [0, tau]
    ind = (t - tv >= 0) & (t - tv <= tau)
    
    # Zero out values outside the active pulse window
    CW_pulse[(t - tv) < 0] = 0
    CW_pulse[(t - tv) > tau] = 0
    
    if np.any(ind):
        # Apply the Tukey window
        window = tukey(np.sum(ind), win)
        CW_pulse[ind] *= window
    
    return CW_pulse


In [9]:
def RCoeff(frequency, theta_i_degrees, cw, cp1, rho_ratio, rhow, alphap1):
    """
    Function to compute reflection coefficient.
    """
    # Convert attenuation to loss parameter
    deltap1 = alphap1 * cp1 * np.log(10) / (40 * np.pi * frequency)

    # Convert from degrees to radians
    theta_i = np.deg2rad(theta_i_degrees)

    # Complex sound speed
    cpp = cp1 / (1 + 1j * deltap1)

    # Ratio of sound speeds
    ap = cpp / cw

    # Impedance for water
    zw = rhow * cw / (np.abs(np.sin(theta_i)) + 1e-6)

    # Impedance of sediment
    zp = rho_ratio * cpp / np.sqrt(1 - (ap * np.cos(theta_i))**2)

    # Impedance ratio
    zwp = zp / zw

    # Reflection coefficient
    R1 = (zwp - 1) / (zwp + 1)

    return R1

In [10]:
def Absorption(frequency, T_C, P_atm, Water_Type):
    """
    Returns absorption (dB/m) of freshwater or seawater of salinity ~35 ppt, given the following inputs:

    frequency = frequency (Hz)
    T_C = temperature (°C) valid range: 0 <= T_C <= 30
    P_atm = hydrostatic pressure (atm) valid range: 1 <= P_atm <= 400
    Water_Type = "Freshwater" or "Seawater"

    Source: Kinsler, Frey, Coppens, and Sanders. Fundamentals of 
    Acoustics, 3rd Ed.  Pages 158 through 160.  Model reprinted from
    Fisher and Simmons.  J. Acoust. Soc. Am 62, 558, 1977. 
    """

    f1 = 1320 * (T_C + 273.15) * np.exp(-1700 / (T_C + 273.15))
    f2 = (1.55e+7) * (T_C + 273.15) * np.exp(-3052 / (T_C + 273.15))

    A = 8.95e-8 * (1 + 0.023 * T_C - 5.1e-4 * T_C**2)
    B = 4.88e-7 * (1 + 0.013 * T_C) * (1 - 0.09e-3 * P_atm)
    C = 4.76e-13 * (1 - 0.04 * T_C + 5.9e-4 * T_C**2) * (1 - 3.8e-4 * P_atm)

    if Water_Type == "Freshwater":

        a = C * frequency**2

    else:

        a = (A * f1 * frequency**2) / (f1**2 + frequency**2) + (B * f2 * frequency**2) / (f2**2 + frequency**2) + C * frequency**2

    return a

In [7]:
def simulate_multiple_ray_propagation_ex1(
    sediment_type,
    receiver_range,
    number_rays,
    time_range,
    dB_range,
):
    """
    Simulates multiple ray propagation.
    """
    # Get start and end display times
    display_time_start, display_time_end = time_range

    # Get intensity start and end display values
    display_intensity_start, display_intensity_end = dB_range

    # Frequency (Hz)
    f = 10000

    # Source level (dB)
    source_level = 170

    # Source depth
    source_depth = 5

    # receiver depth
    receiver_depth = 20

    # seafloor depth
    seafloor_depth = 30

    # Where the source starts
    source_range = 0

    # CW pulse cycles
    pulse_width = 4

    # Plot start and end times
    display_time_start = display_time_start
    display_time_end = display_time_end
    
    # Set propagation parameters based on sediment type
    if sediment_type == "Fine Sand":
        cw = 1500
        cp = 1.15 * cw
        rhop = 2.051
        rhow = 1
        deltap = 0.01
    elif sediment_type == "Silty Clay":
        cw = 1500
        cp = 0.982 * cw
        rhop = 1.546
        rhow = 1
        deltap = 0.001
    elif sediment_type == "Clayey Silt":
        cw = 1500
        cp = 1.01 * cw
        rhop = 1.597
        rhow = 1
        deltap = 0.005
    
    # dB/m conversion parameter
    alphap = 40 * pi * f * deltap / (cp * log(10))

    # Wavenumber in water
    k = 2 * pi * f / cw

    # Add absorption
    water_absorption = True
    if water_absorption:
        absorpValue = Absorption(f, 5, 1, "Seawater")
        k = k + 1j * absorpValue / (20 * np.log10(np.exp(1)))
    

    # Calculate rays (in multiples of 4)
    number_raysets = int(ceil(number_rays / 4))

    # Travel times for rays
    t_v_all = []

    # Pressures for rays
    p_v_all = []

    # Ray path coordinates
    rays = []

    # Iterate through ray sets
    for j_r in range(1, number_raysets + 1):
        # Calculate depth differences for virtual sources
        z_v = np.zeros(4)
        z_v[0] = 2 * seafloor_depth * (j_r - 1) + source_depth - receiver_depth
        z_v[2] = 2 * seafloor_depth * j_r - source_depth - receiver_depth
        z_v[1] = -(2 * seafloor_depth * (j_r - 1) + source_depth + receiver_depth)
        z_v[3] = -(2 * seafloor_depth * j_r - source_depth + receiver_depth)
        t_v = np.zeros(4)
        p_v = np.zeros(4, dtype=complex)
        r_v = np.zeros(4)
        r_angle = np.zeros(4)
        rc1 = np.zeros(4, dtype=complex)
        
        for j_v in range(4):
            # Grazing angle
            r_angle[j_v] = atan(abs(z_v[j_v]) / (receiver_range - source_range))

            # Reflection coefficient
            rc1[j_v] = RCoeff(f, np.degrees(r_angle[j_v]), cw, cp, rhop / rhow, rhow, alphap)

            # Ray travel distance
            r_v[j_v] = sqrt((receiver_range - source_range)**2 + z_v[j_v]**2)

            # Travel time
            t_v[j_v] = r_v[j_v] / cw
        
        # Calculate pressure for each ray
        p_v[0] = ((-rc1[0])**(j_r - 1)) * np.exp(1j * k * r_v[0]) / r_v[0]
        p_v[1] = -((-rc1[1])**(j_r - 1)) * np.exp(1j * k * r_v[1]) / r_v[1]
        p_v[2] = ((-rc1[2])**(j_r - 1)) * rc1[2] * np.exp(1j * k * r_v[2]) / r_v[2]
        p_v[3] = -((-rc1[3])**(j_r - 1)) * rc1[3] * np.exp(1j * k * r_v[3]) / r_v[3]
        
        # Determine intersections with the surface/seafloor

        # Slope of the ray line
        m_v = -z_v / (receiver_range - source_range)

        # Intercept
        b_v = receiver_depth - m_v * receiver_range
        
        # Calculate intersection levels
        l1 = np.arange(2 * (j_r - 1), 0, -1) * seafloor_depth
        l3 = np.arange(2 * j_r - 1, 0, -1) * seafloor_depth
        if j_r - 1 > 0:
            l2 = -np.append(np.arange(2 * (j_r - 1), 0, -1) * seafloor_depth, 0)
        else:
            l2 = np.array([0])
        l4 = -np.append(np.arange(2 * j_r - 1, 0, -1) * seafloor_depth, 0)
        
        # Compute intersections if the slope is nonzero
        if m_v[0] != 0 and l1.size > 0:
            r_i1 = (l1 - b_v[0]) / m_v[0]
            z_i1 = np.tile(np.array([0, seafloor_depth]), j_r - 1) if j_r - 1 > 0 else np.array([])
        else:
            r_i1, z_i1 = np.array([]), np.array([])

        if m_v[1] != 0 and l2.size > 0:
            r_i2 = (l2 - b_v[1]) / m_v[1]
            z_i2 = np.concatenate(([0], np.tile(np.array([seafloor_depth, 0]), j_r - 1)))
        else:
            r_i2, z_i2 = np.array([]), np.array([])

        if m_v[2] != 0 and l3.size > 0:
            r_i3 = (l3 - b_v[2]) / m_v[2]
            z_i3 = np.concatenate(([seafloor_depth], np.tile(np.array([0, seafloor_depth]), j_r - 1)))
        else:
            r_i3, z_i3 = np.array([]), np.array([])

        if m_v[3] != 0 and l4.size > 0:
            r_i4 = (l4 - b_v[3]) / m_v[3]
            z_i4 = np.tile(np.array([seafloor_depth, 0]), j_r)
        else:
            r_i4, z_i4 = np.array([]), np.array([])

        # Save ray paths
        rays.append({"ray_r": np.concatenate(([source_range], r_i1, [receiver_range])),
                     "ray_z": np.concatenate(([source_depth], z_i1, [receiver_depth]))})
        rays.append({"ray_r": np.concatenate(([source_range], r_i2, [receiver_range])),
                     "ray_z": np.concatenate(([source_depth], z_i2, [receiver_depth]))})
        rays.append({"ray_r": np.concatenate(([source_range], r_i3, [receiver_range])),
                     "ray_z": np.concatenate(([source_depth], z_i3, [receiver_depth]))})
        rays.append({"ray_r": np.concatenate(([source_range], r_i4, [receiver_range])),
                     "ray_z": np.concatenate(([source_depth], z_i4, [receiver_depth]))})
        
        # Save travel times and pressures
        t_v_all.extend(t_v.tolist())
        p_v_all.extend(p_v.tolist())
    
    # Time series calculation

    # Time step (T/10)
    dt = (1 / f) / 10
    # tmax = max(t_v_all) * 1.5;
    tmax = 5
    t1 = np.arange(0, tmax, dt)
    tau = pulse_width / f      

    # Create CW pulse with a 20% Tukey window taper
    output_pulse = np.zeros_like(t1, dtype=complex)
    for j in range(min(number_rays, len(p_v_all))):
        output_pulse += CW_pulse_propagation(p_v_all[j], t1, t_v_all[j], f, tau, 0.2)

    # Create background noise
    noise_level = 60
    sigma1 = 10**(noise_level / 20)
    background_noise = np.random.normal(0,sigma1,len(t1))

    print(10*np.log10(np.mean(background_noise**2)))
    
    # Create figure containing three subplots
    _, axes = plt.subplots(3, 1, figsize=(18, 15))

    # Create ray paths plot
    for j in range(min(number_rays, len(rays))):
        axes[2].plot(rays[j]["ray_r"], rays[j]["ray_z"], "k")
    axes[2].set_xlim(source_range, receiver_range)
    axes[2].set_ylim(0, seafloor_depth)
    axes[2].set_xlabel("Range (m)")
    axes[2].set_ylabel("Depth (m)")
    axes[2].set_title("Ray Paths")
    axes[2].grid(True)
    axes[2].invert_yaxis()

    # Create pressure time series plot
    axes[1].plot(t1, background_noise + np.real(output_pulse) * np.sqrt(2) * 10**(source_level / 20), linewidth=0.3)
#     axes[1].set_ylim(-0.01, 0.01)
    axes[1].set_xlim(display_time_start, display_time_end)
    axes[1].set_xlabel("Time (s)")
    axes[1].set_ylabel("Pressure (µPa)")
    axes[1].set_title("Pressure Time Series")
    axes[1].grid(True)

    # Create intensity time series plot
    intensity = 20 * np.log10(np.abs(output_pulse * 10**(source_level / 20) + background_noise))
    axes[0].plot(t1, intensity, linewidth=2)
    axes[0].set_ylim(display_intensity_start, display_intensity_end)
    axes[0].set_xlim(display_time_start, display_time_end)
    axes[0].set_xlabel("Time (s)")
    axes[0].set_ylabel("Intensity (dB)")
    axes[0].set_title("Intensity Time Series")
    axes[0].grid(True)

    # Show plots
    plt.tight_layout()
    plt.show()


# Create interactive widgets
layout = widgets.Layout(width="500px", height="30px")
_ = widgets.interact(
    simulate_multiple_ray_propagation_ex1,
    sediment_type=widgets.Dropdown(
        options=["Fine Sand", "Silty Clay", "Clayey Silt"],
        value="Fine Sand",
        description="Sediment Type",
        style={"description_width": "initial"}, 
        layout=layout,
    ),
    receiver_depth=widgets.IntSlider(
        min=0,
        max=100,
        step=1,
        value=20,
        description="Receiver Depth (m)",
        style={"description_width": "initial"},
        layout=layout,
        continous_update = False,
    ),
    receiver_range=widgets.IntSlider(
        min=1,
        max=500,
        step=10,
        value=100,
        description="Receiver Range (m)",
        style={"description_width": "initial"},
        layout=layout,
        continous_update = False,
    ),
    number_rays=widgets.IntSlider(
        min=4,
        max=100,
        step=4,
        value=10,
        description="Number of Rays",
        style={"description_width": "initial"},
        layout=layout,
        continous_update = False,
    ),
    time_range=widgets.FloatRangeSlider(
        value=[0, 0.5],
        min=0,
        max=0.5,
        step=0.01,
        description="Time Range (s):",
        style={"description_width": "initial"},
        layout=layout,
        continous_update = False,
    ),
    dB_range=widgets.FloatRangeSlider(
        value=[55, 150],
        min=0,
        max=200,
        step=1,
        description="Intensity Range (dB):",
        style={"description_width": "initial"},
        layout=layout,
        continous_update = False,
    ),
)


interactive(children=(Dropdown(description='Sediment Type', layout=Layout(height='30px', width='500px'), optio…